<h2>Привет!</h2>


Напишу пару слов про свой подход:
Поскольку я делал много проектов по NLP то много раз работал с текстовыми данными и пошел я собственно по этому пути.




Данные для обучения:


*  **Фильмы.** Я взял все теги для фильма и плюс добавил его рейтинг. Название фильма, год и жанры я в итоговой реализации проигнорировал, хотя последние два признака я думаю помогут сильно повысить качество в итоге.
*  **Пользователь.** Я взял все теги пользователя и фильмы 50 самых популярных тегов которые ему понравились.


Получается что в каждом случае это строка состоящая из тегов. Я ограничился их количество до 128.


Далее получил для каждого тега его input_id из предобученного токенайзера от Bert.


**Разделение на train и test**
Я поступил достаточно просто и для обучения и тестирования взял 12 тысяч наблюдений. 80% я отдал на обучение. Я ограничился train и val выборкой поскольку времени слишком мало на адекватное тестирование, также как и на масштабную тренировку. При этом при разделении я взял 6 тысяч наблюдений где рейтинг ниже 2.5 и 6 где больше. Баланс классов сильно поможет при обучении в моем подходе.


**Модель**
Взял конечно же предобученный трансформер - классический Bert. А точнее из него я беру эмбеддинг для слов из скрытого состояния. А затем получившийся embedding для человека и фильма я прогоняю в линейный слой, по сути это проекция на меньший масштаб, в моем случае вектор на 512 значений.


**Метрика**
В качестве метрики я использовал cosine similarity. В моем случае если человек поставил фильму оценку меньше 2.5 то фильм плохой и не понравился ему. Значит расстояние между таким человеком и вектором фильма должно быть меньше. В противном случае он отнесся нейтрально к фильму или он ему понравился - тогда их близость должна расти. В качестве loss функции была Cosine Embedding Loss. Также косинусное расстояние. Косинусное расстояние является хорошим выбором для данной задачи рекомендации фильмов по тегам, так как оно является метрикой сходства между векторами, которая не зависит от их длины и может быть использована для сравнения векторов разной размерности. С точки зрения представления пользователь и фильмы, понравившиеся ему должны быть напралвены в одном направлении, а задизлайканные удаляться в ином направлении, в сторону. 


In [ ]:
!wget https://files.grouplens.org/datasets/movielens/ml-latest.zip
!unzip /content/ml-latest.zip

--2023-05-10 18:41:26--  https://files.grouplens.org/datasets/movielens/ml-latest.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277113433 (264M) [application/zip]
Saving to: ‘ml-latest.zip’

ml-latest.zip       100%[===================>] 264.28M  28.9MB/s    in 10s     

2023-05-10 18:41:36 (26.6 MB/s) - ‘ml-latest.zip’ saved [277113433/277113433]

Archive:  /content/ml-latest.zip
   creating: ml-latest/
  inflating: ml-latest/links.csv     
  inflating: ml-latest/tags.csv      
  inflating: ml-latest/genome-tags.csv  
  inflating: ml-latest/ratings.csv   
  inflating: ml-latest/README.txt    
  inflating: ml-latest/genome-scores.csv  
  inflating: ml-latest/movies.csv    


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 82.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 119.0 MB/s eta 0:00:00


In [ ]:
import re
import torch
import pandas as pd
import numpy as np

from tqdm.notebook import trange, tqdm
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from torch import nn 

import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

In [ ]:
movies_df = pd.read_csv('/content/ml-latest/movies.csv', index_col='movieId')
ratings_df = pd.read_csv('/content/ml-latest/ratings.csv')
tags_df = pd.read_csv('/content/ml-latest/tags.csv')

Напишем несколько функций для удобной работы с данными

In [ ]:
def create_user_df(tags_df: pd.DataFrame, ratings_df: pd.DataFrame, num_popular_tags=50) -> pd.DataFrame:
    """
    Функция которая создает новый dataframe для всех пользоватлей в ratings_df
    В нем есть колонка с id пользователя и его тегами + num_popular_tags самых 
    популярных фильмов, которые ему понравились. 
    """
    
    # Загружаем уникальных пользователей
    user_movie_list = ratings_df[['userId', 'movieId']].groupby('userId')['movieId'].apply(list).reset_index(name='movies')
    tag_counts = tags_df['tag'].value_counts().reset_index(name='count').rename(columns={'index': 'tag'})
    user_df = pd.DataFrame(columns=['userId', 'tags'])
    
    
    for _, row in user_movie_list.iterrows():
        
        # Выбираем самые популярные теги для фильмов понравившихся пользователю
        user_tags = tags_df.loc[tags_df['movieId'].isin(row['movies']), 'tag']
        popular_tags = tags_df.loc[tags_df['movieId'].isin(row['movies'])].groupby('movieId')['tag'] \
                              .apply(lambda x: x.value_counts().nlargest(num_popular_tags).index.tolist()).explode().unique()
        
        user_df = user_df.append({'userId': row['userId'], 'tags': list(set(user_tags) | set(popular_tags))}, ignore_index=True)
        
    return user_df

In [ ]:
def transform_movies_df(movies_df: pd.DataFrame, tags_df: pd.DataFrame, num_popular_tags=50) -> pd.DataFrame:
    """
    Функция которая создает новый dataframe для всех фильмов в movies_df
    В нем есть колонка с id фильма и его тегами.
    """
    
    new_movies_df = pd.DataFrame(columns=['movieId', 'tags'])

    for movie in movies_df.index:
        popular_tags = []
        popular_tags.append(ratings_df[ratings_df['movieId'] == movie]['rating'].mean())
        popular_tags.extend(list(tags_df[tags_df['movieId'] == movie]['tag'].value_counts().index)[:num_popular_tags])
        
        new_movies_df = new_movies_df.append({'movieId': movie, 'tags': popular_tags}, ignore_index=True)
        
    return new_movies_df

Пример работы функции

In [ ]:
 transform_movies_df(movies_df[:1000], tags_df)

,movieId,tags
0,1,"[3.8866494325899312, animation, Pixar, pixar, ..."
1,2,"[3.246582912721512, Robin Williams, fantasy, t..."
2,3,"[3.173981392364453, moldy, fishing, Jack Lemmo..."
3,4,"[2.874539979926397, chick flick, characters, g..."
4,5,"[3.0772909396406876, steve martin, pregnancy, ..."
...,...,...
995,1014,"[3.377006880733945, Hayley Mills, Disney, peri..."
996,1015,"[3.2792029317453046, family, nostalgic, feel-g..."
997,1016,"[3.1331821127673365, family, dog, magic, sheep..."
998,1017,"[3.506875168509032, island, shipwreck, strande..."


Датасет по которому мы будем создавать обучающую и тестовую выборку

In [ ]:
class MovieLensDataset(Dataset):
    """
    Класс датасета
    max_length - максимальное количество слов
    """
    def __init__(self, ratings_df, movies_df, tags_df, n_tags=50, max_length=128, model_name='bert-base-uncased'):
        self.ratings_df = ratings_df
        self.movies_df = movies_df[movies_df.index.isin(ratings_df['movieId'])]
        self.tags_df = tags_df[tags_df['movieId'].isin(ratings_df['movieId'])]

        self.user_df = create_user_df(self.tags_df, self.ratings_df)
        self.film_df = transform_movies_df(self.movies_df, self.tags_df)

        self.tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
        self.max_length = max_length

    def __len__(self):
        return len(self.ratings_df)

    def __getitem__(self, index):
        # Выбираем пользователя и фильм из ratings_df
        user_id = self.ratings_df.iloc[index]['userId']
        user_tags = self.user_df.loc[self.user_df['userId'] == user_id, 'tags'].values
        movie_id = self.ratings_df.iloc[index]['movieId']
        movie_tags = self.film_df.loc[self.film_df['movieId'] == movie_id, 'tags'].values
        
        user_str = ' '.join([str(x) for x in user_tags[0]])
        movie_str = ' '.join([str(x) for x in movie_tags[0][1:]])

        # Отдельно обрабатываем рейтинг, в модель он пойдет отдельно
        mean_movie_rating = torch.tensor(float(movie_tags[0][0]))
        rating = self.ratings_df.iloc[index]['rating']
        
        # Токенезируем все теги
        user_tokens = self.tokenizer(user_str, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')
        movie_tokens = self.tokenizer(movie_str, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')

        user_input_ids = user_tokens['input_ids'].squeeze()
        user_attention_mask = user_tokens['attention_mask'].squeeze()
        movie_input_ids = movie_tokens['input_ids'].squeeze()
        movie_attention_mask = movie_tokens['attention_mask'].squeeze()
        
        return user_input_ids, user_attention_mask, movie_input_ids, movie_attention_mask, rating, mean_movie_rating

В моей реаилзации я ограничился 12 тысячами, так уже при таких небольших цифрах(по сравнению со всем датасетом) обрбаотка в моем подходе занимет 6 минут. Данные выбраны так, чтобы в них были как плохие оценки, так и хорошие в равной пропорции

In [ ]:
%%time

# Берем поизитинвые и негативные оценки
positive_ratings = ratings_df[ratings_df['rating'] >= 2.5]
negative_ratings = ratings_df[ratings_df['rating'] < 2.5]

num_samples = 6000  # Количетсво наблюдений для фильмов, которые понравились или нет
positive_samples = positive_ratings.sample(num_samples)
negative_samples = negative_ratings.sample(num_samples)

balanced_ratings = pd.concat([positive_samples, negative_samples])
balanced_ratings = balanced_ratings.sample(frac=1).reset_index(drop=True)
dataset = MovieLensDataset(balanced_ratings, movies_df, tags_df)

# Разделяем данные на два dataset
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

CPU times: user 6min 23s, sys: 1.36 s, total: 6min 24s
Wall time: 5min 12s


Сама модель. Берт и два линейных слоя для проекции на меньшую размерность

In [ ]:
class MovieLensModel(nn.Module):
    def __init__(self, embedding_dim=512):
        super(MovieLensModel, self).__init__()

        # Предобученный Bert
        self.bert_model = AutoModel.from_pretrained("bert-base-uncased")

        # Два лсоя с проекциями, из которых мы получаем embedding
        self.user_projection = nn.Linear(self.bert_model.config.hidden_size, embedding_dim)

        # добавляем 1 - средний рейтинг фильма
        self.movie_projection = nn.Linear(self.bert_model.config.hidden_size + 1, embedding_dim)  

    def forward(self, user_input_ids, user_attention_mask, movie_input_ids, movie_attention_mask, mean_rating):

        # Получаем предобученные embeddings из Берта для фильмов и пользователей.
        user_outputs = self.bert_model(user_input_ids, attention_mask=user_attention_mask)
        user_embeddings = user_outputs.last_hidden_state[:, 0, :]
        movie_outputs = self.bert_model(movie_input_ids, attention_mask=movie_attention_mask)
        movie_embeddings = movie_outputs.last_hidden_state[:, 0, :]

        # обрабтываем средний рейтинг и получаем большой эмбединг фильма
        mean_rating = mean_rating.unsqueeze(-1)  
        movie_embeddings = torch.cat([movie_embeddings, mean_rating], dim=-1)

        # делаем проекции на маленькие пространства
        user_embed = self.user_projection(user_embeddings)
        movie_embed = self.movie_projection(movie_embeddings)

        return user_embed, movie_embed

In [ ]:
model = MovieLensModel().cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Сразу напишем тетсвую функцию, чтобы посмотреть на результат на необученной модели

In [ ]:
def test(model, val_dataset, batch_size=32):
    """
    Тестирующая функция. На выходе получаеться среднее косинусное расстояние между 
    пользователем и фильмами, которые ему понравились, и фильмами, которые ему понравились.
    """
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
    model.eval()

    with torch.no_grad():
        with tqdm(total=len(val_loader), desc="Validation") as progress_bar:
            ratings = []
            user_embeddings = []
            movie_embeddings = []

            for user_input_ids, user_attention_mask, movie_input_ids, movie_attention_mask, rating, mean_rating in val_loader:
                user_input_ids = user_input_ids.cuda()
                user_attention_mask = user_attention_mask.cuda()
                movie_input_ids = movie_input_ids.cuda()
                movie_attention_mask = movie_attention_mask.cuda()
                rating = rating.cuda()
                mean_rating = mean_rating.cuda()

                user_embed, movie_embed = model(user_input_ids, user_attention_mask, 
                                                movie_input_ids, movie_attention_mask, mean_rating)

                user_embeddings.append(user_embed)
                movie_embeddings.append(movie_embed)
                ratings.append(rating)

                progress_bar.update(1)

        user_embeddings = torch.cat(user_embeddings, dim=0)
        movie_embeddings = torch.cat(movie_embeddings, dim=0)
        ratings = torch.cat(ratings, dim=0)

        cosine_sim = nn.functional.cosine_similarity(user_embeddings, movie_embeddings)

        positive_mask = ratings >= 2.5
        negative_mask = ratings < 2.5

        positive_cosine_sim = cosine_sim[positive_mask]
        negative_cosine_sim = cosine_sim[negative_mask]

        positive_mean_cosine_sim = positive_cosine_sim.mean().item()
        negative_mean_cosine_sim = negative_cosine_sim.mean().item()

        print(f"Positive Mean Cosine Similarity: {positive_mean_cosine_sim:.4f}")
        print(f"Negative Mean Cosine Similarity: {negative_mean_cosine_sim:.4f}")

Рузльтат очевиден, свзяи между векторами пользователей и фильмов нет никакой

In [ ]:
test(model, val_dataset)

Validation:   0%|          | 0/75 [00:00<?, ?it/s]

Positive Mean Cosine Similarity: 0.0451
Negative Mean Cosine Similarity: 0.0434


Цикл для обучения и валидации

In [ ]:
def train(model, train_dataset, val_dataset, batch_size=32, lr=1e-4, epochs=4):
    """
    Обучающий цикл:
    Оптимизатор Adam - лушчий вариант, когда не знаешь какой больше подойдет в плане специфики задачи
    lr - базовое значение, без уменьшения с каждой итерацией.
    """

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.CosineEmbeddingLoss()

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        with tqdm(total=len(train_loader), desc="Batch progress") as progress_bar:
          for user_input_ids, user_attention_mask, movie_input_ids, movie_attention_mask, rating, mean_rating in train_loader:
              user_input_ids = user_input_ids.cuda()
              user_attention_mask = user_attention_mask.cuda()
              movie_input_ids = movie_input_ids.cuda()
              movie_attention_mask = movie_attention_mask.cuda()
              rating = rating.cuda()
              mean_rating = mean_rating.cuda()

              optimizer.zero_grad()

              user_embed, movie_embed = model(user_input_ids, user_attention_mask, 
                                              movie_input_ids, movie_attention_mask, mean_rating)

              target = torch.ones_like(rating) 
              target[rating < 2.5] = -1 

              loss = loss_fn(user_embed, movie_embed, target)
              loss.backward()
              optimizer.step()

              train_loss += loss.item() * rating.size(0)
              progress_bar.update(1)

        train_loss /= len(train_dataset)

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
              for user_input_ids, user_attention_mask, movie_input_ids, movie_attention_mask, rating, mean_rating in val_loader:
                  user_input_ids = user_input_ids.cuda()
                  user_attention_mask = user_attention_mask.cuda()
                  movie_input_ids = movie_input_ids.cuda()
                  movie_attention_mask = movie_attention_mask.cuda()
                  rating = rating.cuda()
                  mean_rating = mean_rating.cuda()

                  user_embed, movie_embed = model(user_input_ids, user_attention_mask, 
                                                movie_input_ids, movie_attention_mask, mean_rating)

                  target = torch.ones_like(rating)
                  target[rating < 2.5] = -1

                  loss = loss_fn(user_embed, movie_embed, target)
                  val_loss += loss.item() * rating.size(0)
                  

        val_loss /= len(val_dataset)

        print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
        torch.save(model.state_dict(), '/content/drive/MyDrive//model_embeddings.pt')

    return model

Обучим модель

In [ ]:
model_fitted = train(model, dataset, val_dataset)

Batch progress:   0%|          | 0/375 [00:00<?, ?it/s]

Epoch 1/4, Train Loss: 0.4890, Val Loss: 0.4828


Batch progress:   0%|          | 0/375 [00:00<?, ?it/s]

Epoch 2/4, Train Loss: 0.4548, Val Loss: 0.4383


Batch progress:   0%|          | 0/375 [00:00<?, ?it/s]

Epoch 3/4, Train Loss: 0.4159, Val Loss: 0.3953


Batch progress:   0%|          | 0/375 [00:00<?, ?it/s]

Epoch 4/4, Train Loss: 0.3990, Val Loss: 0.3851


Посмтрим насколько научилась наша моедль понимать пользоватлей и фильмы. Как мы помним вектора до обучения были случайными

In [ ]:
test(model, val_dataset)

Validation:   0%|          | 0/75 [00:00<?, ?it/s]

Positive Mean Cosine Similarity: 0.5751
Negative Mean Cosine Similarity: 0.2436


Как мы видим близость между пользователем и фильмами, которые ему понравились значетльно выше чем между ним и фильмами которым он поставил низкие оценки. 

Благодаря таким резлульатам можно рекомендовать фильмы, так как среднее косинусное расстояния пользователя значительно ближе к фильмам, которые ему нравяться. При этом тут осталось место для крутых улучшений, например у нас есть данные о релевантности тегов для фильмов. Это очень крутой способ построить attention для слов и тем самым повысить качество. Плюс можно увеличит выборку и например усложнить проекцию. Не делать линейный слой, а например пробежаться conv1 взять maxpool и затем уже линейные слои. Я думаю все это позволит значительно поднять качесвто. Данный подход мне нравится поскольку мы не фокусируемся на тегах и не далаем огрмоную разряженную матрицу, а смотрим исключетельно на теги и их смылс, что позволяет нам обрабтывать новые значения, которые не были в train выборку(из за предобученного Bert). Спасибо, что дочитали до сюда, надеюсь я не сильно опозорился, удачи вам)